<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Prediction_EN_791112_50000_51000_below_04281630_thr03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Error: gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Ignore this error as gcsfs is not used.

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
#!pip install -q scikit-learn
!pip install -q transformers datasets  # 2 Hugging Face libraries
#!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy             as np
import os
import pandas            as pd
import shutil
import torch

from contextlib                     import suppress
from datasets                       import DatasetDict
from google.colab                   import files, userdata
from huggingface_hub                import create_repo, hf_hub_download, login, whoami
from sklearn.metrics                import classification_report
from transformers.models.longformer import LongformerTokenizerFast, LongformerForSequenceClassification
from torch.utils.data               import DataLoader

## TO DO BEFORE RUN
- GIVE HF HUB AND W&B ACCESSES
- ADAPT HF HUB MODEL ID

## Hugging Face Hub authenticate

In [3]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"

print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


## Configuration

###### Define literal data

In [4]:
# HF Hub repo
repo_id = "claudelepere/jobs_EN_791112_50000_below_04281630"

skills           = 791112

all_rows_low_trEvTe  = 0
all_rows_high_trEvTe = 50000

all_rows_low     = 50000
all_rows_high    = 51000
num_datapoints   = all_rows_high - all_rows_low

max_length       = 1024
batch_size       = 4*8

threshold_tuning = True

min_threshold    = 0.3
min_trEvTe_count = 50

###### Define input paths

In [5]:
# datasetDict zip file
datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_below.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")

# Optimized thresholds json file
optimized_thresholds_json = hf_hub_download(repo_id=repo_id, filename="optimized_thresholds.json", cache_dir="./my_HF_downloads")
print(f"file_path: {optimized_thresholds_json}")

print(f"cwd: {os.getcwd()}")

pr_below_count_json     = f"pr_{all_rows_low}_{all_rows_high}_below_count.json"
trEvTe_below_count_json = f"trEvTe_{all_rows_low_trEvTe}_{all_rows_high_trEvTe}_below_count.json"

datasetDict_zip_file_name: dataset_EN_791112_50000_51000_below.zip
datasetDict_dir_name     : dataset_EN_791112_50000_51000_below


optimized_thresholds.json:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

file_path: ./my_HF_downloads/models--claudelepere--jobs_EN_791112_50000_below_04281630/snapshots/8768833c5721f3025397dc7c66d4bdafdd3e6b0f/optimized_thresholds.json
cwd: /content


###### Define output paths

In [6]:
# true_labels - preds comparison file
compare_true_labels_preds_file = "compare_true_labels_preds.txt"

## Upload

In [7]:
# Upload the file pr_below_count_json and save it in /content, the current working directory
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")

with open(pr_below_count_json, 'r') as json_file:
    both_dict = json.load(json_file)

pr_counter_dict        = both_dict["counter_dict"]
pr_sorted_counter_dict = both_dict["sorted_counter_dict"]
print(f"pr_counter_dict: {type(pr_counter_dict)} len={len(pr_counter_dict)}\n{json.dumps(pr_counter_dict, indent=4)}")

raise Exception("ZZZZ")

Saving dataset_EN_791112_50000_51000_below.zip to dataset_EN_791112_50000_51000_below.zip
Saving pr_50000_51000_below_count.json to pr_50000_51000_below_count.json
Saving trEvTe_0_50000_below_count.json to trEvTe_0_50000_below_count.json
Uploaded file: dataset_EN_791112_50000_51000_below.zip
Uploaded file: pr_50000_51000_below_count.json
Uploaded file: trEvTe_0_50000_below_count.json
pr_counter_dict: <class 'dict'> len=94
{
    "143": 15,
    "152": 3,
    "155": 10,
    "167": 10,
    "173": 12,
    "175": 17,
    "176": 7,
    "360": 6,
    "370": 1,
    "371": 6,
    "373": 11,
    "375": 10,
    "376": 12,
    "667": 1,
    "686": 13,
    "687": 7,
    "756": 3,
    "758": 3,
    "760": 4,
    "900": 0,
    "901": 0,
    "902": 0,
    "903": 0,
    "905": 0,
    "907": 0,
    "908": 1,
    "909": 0,
    "911": 0,
    "912": 0,
    "913": 0,
    "914": 0,
    "916": 0,
    "263": 9,
    "265": 7,
    "266": 3,
    "267": 2,
    "268": 7,
    "270": 0,
    "272": 14,
    "273": 3,
  

Exception: ZZZZ

## Checks

In [ ]:
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # cuda = A100 GPU or L4 GPU
print(f"device: {device}")

## Out Of Memory (OOM)

### OOM: check for and kill zombie processes

In [ ]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
    !nvidia-smi
    print(torch.cuda.memory_summary())

### OOM: free up GPU memory

In [ ]:
torch.cuda.empty_cache()

## Upload and unzip job dataset

In [ ]:
def upload_unzip_dataset(filename):
    """Upload and unzip the dataset to /content, ensuring correct placement."""

    # Get the expected directory name (same as the zip filename without extension)
    expected_dir = os.path.splitext(filename)[0]

    # Check if the file and the directory exist in /content and delete them
    with suppress(FileNotFoundError):
        if os.path.isdir(expected_dir):
            shutil.rmtree(expected_dir)               # Remove directory if it exists
        if os.path.isfile(filename):
            os.remove(filename)                       # Remove file if it exists

    print(f"Removed '{expected_dir}' and '{filename}' if they were present in /content.")

    # Upload the zip file
    uploaded_files = files.upload()                  # Prompt file upload dialog

    if filename not in uploaded_files:
        raise FileNotFoundError(f"'{filename}' was not uploaded.")

    print(f"'{filename}' successfully uploaded to /content.")

    # Unzip the file to /content
    shutil.unpack_archive(filename, "/content")

    print(f"Unzipped to '/content/{expected_dir}'.")

# Usage
upload_unzip_dataset(datasetDict_zip_file_name)

## Create datasetDict (HF DatasetDict) = 1 HF Dataset prediction
The sequence of datapoints of dataset=datasetDict['prediction'] is the same as the sequence of SELECT, id, ... WHERE in_LanguageId=1 ORDER BY id DESC in MySQL.

In [ ]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)
dataset     = datasetDict['prediction']

print(f"dataset: {type(dataset)} shape={dataset.shape}\n{dataset}")     # <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
print(f"dataset.features: {type(dataset.features)} shape={dataset.features}\n{dataset.features}")

# Convert the dataset to a pandas dataframe
df_original = pd.DataFrame(dataset)

pd.set_option('display.max_columns', None)
print(f"df_original: {type(df_original)} shape={df_original.shape}\n{df_original}")                         # <class 'pandas.core.frame.DataFrame'> shape=(100, 8)
pd.reset_option('display.max_columns')


## Create labels (list), id2label (dict) and label2id (dict).
**The sequence of the labels list is the same as in dataset.
And the sequences of the optimized thresholds, true labels and predictions are preserved.**

* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [ ]:
labels = [label for label in dataset.features.keys() if label not in ['id', 'text']]
#labels.sort()

print(f"labels: {type(labels)} {len(labels)}\n{labels}")

num_labels = len(labels)

In [ ]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

In [ ]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

## Download the tokenizer and the model

In [ ]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id, timeout=60)  # Increased timeout to 60 seconds)

print("Model")
model = LongformerForSequenceClassification.from_pretrained(repo_id)
model.eval()  # Ensures no gradient computation

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Upload the json cout file

In [ ]:
# Upload the file pr_below_count_json and save it in /content, the current working directory
uploaded = files.upload()

# Read the file prediction
with open(pr_below_count_json, 'r') as json_file:
    both_dict = json.load(json_file)

pr_counter_dict        = both_dict["counter_dict"]
pr_sorted_counter_dict = both_dict["sorted_counter_dict"]
print(f"pr_counter_dict: {type(pr_counter_dict)} len={len(pr_counter_dict)}\n{json.dumps(pr_counter_dict, indent=4)}")
print(f"pr_sorted_counter_dict: {type(pr_sorted_counter_dict)} len={len(pr_sorted_counter_dict)}\n{json.dumps(pr_sorted_counter_dict, indent=4)}")

# Upload the file trEvTe_below_count_json and save it in /content, the current working directory
uploaded = files.upload()

# Read the file trEvTe
with open(trEvTe_below_count_json, 'r') as json_file:
    both_dict = json.load(json_file)
trEvTe_counter_dict        = both_dict["counter_dict"]
trEvTe_sorted_counter_dict = both_dict["sorted_counter_dict"]
print(f"trEvTe_counter_dict: {type(trEvTe_counter_dict)} len={len(trEvTe_counter_dict)}\n{json.dumps(trEvTe_counter_dict, indent=4)}")
print(f"trEvTe_sorted_counter_dict: {type(trEvTe_sorted_counter_dict)} len={len(trEvTe_sorted_counter_dict)}\n{json.dumps(trEvTe_sorted_counter_dict, indent=4)}")

#raise Exception("AAAA")

## Downloads the optimized thresholds

- count_0_list = ["900", "901", "902", "903", "905", "907", "909", "911", "912", "913", "914", "916"]
  - count = 0 in prediction below
  - count in trEvTE below:    
        "900": 3,
        "901": 2,
        "902": 1,
        "903": 7,
        "905": 5,
        "907": 2,
        "908": 5,
        "909": 1,
        "911": 6,
        "912": 1,
        "913": 5,
        "914": 3,
        "916": 4,
- count_01_025_list = ["908", "913", "672", "673"]
  - first normalized_thresholds 908 and 913: 0.1, 672 and 673: 0.25
  - count in prediction below: 908: 1, 913: 0, 672: 3, 673: 1
  - count in trEvTe below: 908: 5, 913: 5, 672: 37, 673: 41


In [ ]:
# Load the JSON file
with open(optimized_thresholds_json, 'r') as json_file:
    optimized_thresholds_json = json.load(json_file)

#print(f"optimized_thresholds_json: {type(optimized_thresholds_json)} {len(optimized_thresholds_json)}")
#print(json.dumps(optimized_thresholds_json, indent=2))

optimized_thresholds = np.array(list(optimized_thresholds_json.values()))
print(f"optimized_thresholds before: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")

#optimized_thresholds = np.maximum(optimized_thresholds, min_threshold)
#print(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")


#count_0_list = ["900", "901", "902", "903", "905", "907", "909", "911", "912", "913", "914", "916"]
#count_01_025_list = ["908", "913", "672", "673"]
#count_both = count_0_list + count_01_025_list

#rare_labels_in_trEvTe = [label for label, count in trEvTe_counter_dict.items() if count < min_trEvTe_count]
#print(f"rare_labels_in_trEvTe: {type(rare_labels_in_trEvTe)} len={len(rare_labels_in_trEvTe)}\n{rare_labels_in_trEvTe}")

for idx, (label, count) in enumerate(pr_counter_dict.items()):
    print(f"{idx}: {label} ->{count}: {optimized_thresholds[idx]:.2f}")
#    #if label in count_both:
#    if label in rare_labels_in_trEvTe:
#        optimized_thresholds[idx] = min_threshold

# Vectorize the update of optimized_thresholds
low_count_indices = [idx for idx, (label, count) in enumerate(trEvTe_counter_dict.items()) if count < min_trEvTe_count]
optimized_thresholds[low_count_indices] = min_threshold

print()

print(f"optimized_thresholds after: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")
#raise Exception("BBBB")

In [ ]:
def format_int_or_NaN(x):
    return int(x) if not np.isnan(x) else "NaN"

In [ ]:
def calculate_confusion_multi(precision, recall, support, total_samples):
    precision     = np.array(precision)
    recall        = np.array(recall)
    support       = np.array(support)
    total_samples = np.array(total_samples)

    with np.errstate(divide='ignore', invalid='ignore'):
        f1_score = 2 * (precision * recall) / (precision + recall)
        TP       = recall * support
        FN       = support - TP
        FP       = TP * (1 - precision)/precision
        TN       = total_samples - TP - FN - FP

    return pd.DataFrame({
        'TP' : TP,
        'FN' : FN,
        'FP' : FP,
        'TN' : TN
    })

In [ ]:
def analyze_multi_label_performance(y_true, y_pred, label_names=None, plot=True):
    """
    Returns:
    - label_df    : per-label precision, recall, f1, support (DataFrame)
    - summary_df  : micro, macro, weighted and samples averages (DataFrame)
    - confusion_df: per-label TP, FP, FN, TN (DataFrame)
    """
    report_dict = classification_report(
        y_true, y_pred,
        target_names=label_names,
        zero_division=0,
        output_dict=True
    )
    report_df = pd.DataFrame(report_dict).T
    #print(f"report_df: {type(report_df)} shape={report_df.shape}\n{report_df}")

    # label_df
    label_df = report_df.loc[label_names].copy()
    label_df['support'] = label_df['support'].apply(format_int_or_NaN)
    #print(f"label_df: {type(label_df)} shape={label_df.shape}\n{label_df}")

    precision = label_df['precision'].values
    recall    = label_df['recall'].values
    f1_score  = label_df['f1-score'].values
    support   = label_df['support'].values

    total_samples = len(y_true)
    print(f"total_samples: {type(total_samples)} {total_samples}")  # <class 'int'> 1000
    total_support = label_df['support'].sum()
    print(f"total_support: {type(total_support)} {total_support}")  # <class 'numpy.float64'> 6441.0

    confusion_df       = calculate_confusion_multi(precision, recall, support, total_samples)
    confusion_df.index = label_names
    #confusion_df       = confusion_df.applymap(format_int_or_NaN)
    confusion_df       = confusion_df.apply(lambda col: col.map(format_int_or_NaN))
    #print(f"confusion_df: {type(confusion_df)} shape={confusion_df.shape}\n{confusion_df}")

    # All average rows: micro, macro, weighted, samples
    avg_rows      = ["micro avg", "macro avg", "weighted avg", "samples avg"]
    existing_avgs = [row for row in avg_rows if row in report_df.index]

    summary_df            = report_df.loc[existing_avgs, ["precision", "recall", "f1-score", "support"]]
    summary_df.loc["micro avg", "support"]    = total_support
    summary_df.loc["macro avg", "support"]    = np.nan         # no clear meaning
    summary_df.loc["weighted avg", "support"] = total_support  # number of samples
    summary_df.loc["samples avg", "support"]  = total_samples  # number of samples
    summary_df["support"]                     = summary_df["support"].apply(format_int_or_NaN)
    #print(f"summary_df: {type(summary_df)} shape={summary_df.shape}\n{summary_df}"


    # Optional plot
    if plot:
        #label_df[["precision", "recall", "f1", "support"]].plot(
        label_df[["precision", "recall", "support"]].plot(
            kind="bar", figsize=(10,6), ylim=(0,1),
            title="Per-Label Precision Recall F1 Support"
        )
        plt.grid(axis='y')
        plt.ylabel("Score")
        plt.tight_layout()
        plt.show()

    return label_df, confusion_df, summary_df



## case TP=0 FN!=0 and case TP=0 FN=0

|label|TP|FN|FP|TN|label|precision|recall|f1|support|
|:---:|::|::|::|::|:---:|:-------:|:----:|::|:-----:|
|266|0|3|NaN|NaN|266|0.00|0.00|0.00|3|
|267|0|2|NaN|NaN|267|0.00|0.00|0.00|2|
|273|0|3|NaN|NaN|273|0.00|0.00|0.00|3|
|275|0|0|NaN|NaN|275|0.00|0.00|0.00|0|
|279|0|1|NaN|NaN|279|0.00|0.00|0.00|1|
|285|0|3|NaN|NaN|285|0.00|0.00|0.00|3|
|301|0|0|NaN|NaN|301|0.00|0.00|0.00|0|
|618|0|0|NaN|NaN|618|0.00|0.00|0.00|0|
|681|0|3|NaN|NaN|681|0.00|0.00|0.00|3|
|815|0|1|NaN|NaN|815|0.00|0.00|0.00|1|
|825|0|1|NaN|NaN|825|0.00|0.00|0.00|1|

- TP=0 and FN!=0:
 - precision = 0
 - recall    = 0
 - f1        = 0
 - support   = FN (some true positives to be found)
- TP=FN=0:
 - precision = 0/0 (undefined)
 - recall    = 0/0 (undefined)
 - f1        = 0/0 (undefined)
 - support   = 0   (label never present in the ground truth, the model never predicted it)
- Check using jobs_preprocessing_CounterPrediction:
 - count_275: <class 'int'> 0
 - count_301: <class 'int'> 0
 - count_618: <class 'int'> 0

In [ ]:
# Step 1: Tokenize dataset
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', max_length=max_length, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# <class 'datasets.arrow_dataset.Dataset'> shape=(100, 10)
#print(f"tokenized_dataset: {type(tokenized_dataset)} shape={tokenized_dataset.shape}\n{tokenized_dataset}")

tokenized_df = tokenized_dataset.to_pandas()
#print(f"tokenized_df: {type(tokenized_df)} shape={tokenized_df.shape}\n{tokenized_df}")

# Step 2: Convert to PyTorch DataLoader
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + labels)
dataloader = DataLoader(tokenized_dataset, batch_size=batch_size)

# Step 3: Run Prediction
all_preds = []

with torch.no_grad():                       # No gradients needed for prediction
    for batch in dataloader:
        # Move batch to GPU if available
        inputs = {k: batch[k].to(device) for k in ["input_ids", "attention_mask"]}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Model outputs raw logits

        # Apply sigmoid to convert logits into probabilities
        probs = torch.sigmoid(logits)

        # Move probs to CPU and convert to NumPy
        all_preds.extend(probs.cpu().numpy())

all_preds_arr = np.array(all_preds)
#print(f"all_preds    : {type(all_preds)}     len={len(all_preds)}\n{all_preds}")
print(f"ZOZIZUall_preds_arr: {type(all_preds_arr)} shape={all_preds_arr.shape}\n{all_preds_arr}")
print(f"ZOZIZUall_preds_arr[0]: {all_preds_arr[0]}")

# Step 4: Convert Probabilities to Binary Predictions
if threshold_tuning:
    thresholds = optimized_thresholds
else:
    thresholds = np.full(num_labels, 0.5)
print(f"thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")                  # <class 'numpy.ndarray'> shape=(6,)
print()

binary_preds = (np.array(all_preds) > thresholds).astype(int)
#print(f"binary_preds: {type(binary_preds)} shape={binary_preds.shape}\n{binary_preds}")          # <class 'numpy.ndarray'> shape=(100, 6)
print(f"binary_preds[0]: {binary_preds[0]}")

# Step 5: Compare with True Labels
true_labels = tokenized_df[labels]
#print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")              # <class 'pandas.core.frame.DataFrame'> shape=(100, 6)

# Convert true_labels DataFrame to a NumPy array of 0 and 1
true_labels_np = true_labels.astype(int).to_numpy()
#print(f"true_labels_np: {type(true_labels_np)} shape={true_labels_np.shape}\n{true_labels_np}")  # <class 'numpy.ndarray'> shape=(100, 6)

# Convert id2label dict to a list of label names
label_names = list(id2label.values())
print(f"label_names: {type(label_names)} len={len(label_names)}\n{label_names}")  # <class 'list'> len=126
print()

#classification_report(y_true, y_pred)
report = classification_report(
    true_labels_np, binary_preds, target_names=label_names, zero_division=0)
print(f"classification report:{type(report)} len={len(report)}\n{report}")      # <class 'str'> len=7076
print()

#report2 = classification_report(true_labels_np, binary_preds, target_names=label_names, zero_division=0, output_dict=True)
#print(f"classification report2:{type(report2)} len={len(report2)}\n{report2}")  # <class 'dict'> len=130
#print()

label_df, confusion_df, summary_df = analyze_multi_label_performance(
    true_labels_np, binary_preds, label_names=label_names, plot=False)

label_df[['precision', 'recall', 'f1-score']] = label_df[['precision', 'recall', 'f1-score']].round(2)
label_df['support'] = label_df['support'].astype(int)

summary_df[['precision', 'recall', 'f1-score']] = summary_df[['precision', 'recall', 'f1-score']].round(2)

print("__________________________________________")

print("\nclassification report (= label_df):\n", label_df)

print("__________________________________________")

print("\nAll averages:\n", summary_df)

print("__________________________________________")

#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)
print("\nConfusion matrix:")
print("precision = TP / (TP+FP)")
print("recall    = TP / (TP+FN)")
print("1/f1      = 1/2 * (1/precision + 1/recall")
print("support   = TP+FN")
print("\nConfusion matrix:\n", confusion_df)

print("__________________________________________")

#raise Exception("I stop here")
df_compare                = df_original.copy()
df_compare['true_labels'] = true_labels.values.tolist()
df_compare['preds']       = binary_preds.tolist()
df_compare['compare']     = (df_compare['true_labels']==df_compare['preds']).replace({True:'OK', False:'KO'})
#print(f"df_compare: {type(df_compare)} shape={df_compare.shape}\n{df_compare}")  # <class 'pandas.core.frame.DataFrame'>
print()

def print_row(row, file):
    """Prints the row information to the console and writes it to a file."""
    output  = f"id: {row['id']}\n"
    output += f"text: {row['text']}\n"
    output += "labels     : [" + ", ".join(f"{label:>5}" for label in labels) + "]\n"
    output += "true_labels: [" + ", ".join(f"{true_label:>5}" for true_label in row['true_labels']) + "]\n"
    output += "preds      : [" + ", ".join(f"{pred:>5}" for pred in row['preds']) + "]\n"
    output += f"compare    : {row['compare']}\n"
    output += "\n"

    print(output, end="")  # Print to console without extra newline
    file.write(output)     # Write to file

# Open the file in write mode
with open(compare_true_labels_preds_file, "w") as f:
    _ = df_compare.apply(print_row, axis=1, args=(f,))  # Pass file object to print_row

# Download the file
files.download(compare_true_labels_preds_file)

# Count total 'OK' and 'KO' values
total_ok = (df_compare['compare'] == 'OK').sum()  # Total matches
total_ko = (df_compare['compare'] == 'KO').sum()  # Total mismatches
print(f"Total 'OK': {total_ok}")
print(f"Total 'KO': {total_ko}")
print()

In [ ]:
print("It's the end")

In [ ]:
raise Exception("I stop here")